In [8]:
# CODE implementing ANN for binary classification (with one hidden layer)

# Importing all the required libraries

import os
import numpy as np
import cv2
from sklearn.preprocessing import StandardScaler
from tensorflow import keras
from sklearn.impute import SimpleImputer


# Loading the training data
train_raveling = os.listdir('train/Raveling')
train_non_raveling = os.listdir('train/Non_raveling')
train_images = []
train_labels = []

# preprocess the training data
for img in train_raveling:
    image = cv2.imread('train/Raveling/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w
   
    # Calculating features as given in the paper 

    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)

    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    train_images.append(features)
    train_labels.append(1)  #label raveling as 1

for img in train_non_raveling:
    image = cv2.imread('train/Non_raveling/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w

    # Calculating features as given in the paper 

    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)
    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    train_images.append(features)
    train_labels.append(0)  #label non-raveling as 0
train_images = np.array(train_images)
train_labels = np.array(train_labels) 

# removing NaN error
imp = SimpleImputer(strategy='mean')
train_images = imp.fit_transform(train_images)
scaler = StandardScaler()
train_images = scaler.fit_transform(train_images)


# Building a shallow neural network with a single hidden layer (128 units)

# model = keras.Sequential([keras.layers.InputLayer(input_shape=(train_images.shape[1],)),
#     keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dense(1, activation='sigmoid')])   0.9666 score

model = keras.Sequential([keras.layers.InputLayer(input_shape=(train_images.shape[1],)),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')])

# Compile the model with binary cross-entropy loss, Adam optimizer, and accuracy metric
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model to the training data
model.fit(train_images, train_labels, epochs=350, batch_size=32)


test_images=[]
filenames=[]

# Loading  and preprocessing the test data

for i in range(1, 301):
    image = cv2.imread('test/' + str(i) + '.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w
    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)

    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    test_images.append(features)
    filenames.append(str(i) + '.jpg')

test_images = np.array(test_images)
imp = SimpleImputer(strategy='mean')
test_images = imp.fit_transform(test_images)
test_images = scaler.transform(test_images)
predictions=[]
# Predicting the test labels
predictions = model.predict(test_images)

C:\Users\MOHAMMAD SAIZE ALI\AppData\Local\Temp\ipykernel_13868\2018821962.py:32: RuntimeWarning: divide by zero encountered in log2
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
C:\Users\MOHAMMAD SAIZE ALI\AppData\Local\Temp\ipykernel_13868\2018821962.py:32: RuntimeWarning: invalid value encountered in multiply
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))


Epoch 1/350


C:\Users\MOHAMMAD SAIZE ALI\anaconda3\Lib\site-packages\keras\src\layers\core\input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.6805 - loss: 0.6267
Epoch 2/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7462 - loss: 0.4775
Epoch 3/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7895 - loss: 0.4301
Epoch 4/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7882 - loss: 0.4250
Epoch 5/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7974 - loss: 0.4331
Epoch 6/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8272 - loss: 0.4125
Epoch 7/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8380 - loss: 0.3982
Epoch 8/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8191 - loss: 0.3796
Epoch 9/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8060 - loss: 0.4163
Epoch 10/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8461 - loss: 0.3675
Epoch 11/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8245 - loss: 0.3815
Epoch 12/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8469 - lo

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9189 - loss: 0.1958
Epoch 70/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9437 - loss: 0.1794
Epoch 71/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9139 - loss: 0.2041
Epoch 72/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9356 - loss: 0.1733
Epoch 73/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9341 - loss: 0.1896
Epoch 74/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9151 - loss: 0.2082
Epoch 75/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9323 - loss: 0.1751
Epoch 76/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9408 - loss: 0.1785
Epoch 77/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9405 - loss: 0.1813
Epoch 78/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9362 - loss: 0.1932
Epoch 79/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9441 - loss: 0.1665
Epoch 80/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9

...

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9787 - loss: 0.0865
Epoch 205/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9729 - loss: 0.0938
Epoch 206/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9690 - loss: 0.0990
Epoch 207/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9745 - loss: 0.0853
Epoch 208/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9668 - loss: 0.0930
Epoch 209/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9810 - loss: 0.0865
Epoch 210/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9736 - loss: 0.0944
Epoch 211/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9664 - loss: 0.1022
Epoch 212/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9673 - loss: 0.1050
Epoch 213/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9774 - loss: 0.0834
Epoch 214/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9775 - loss: 0.0869
Epoch 215/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - ac

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.9762 - loss: 0.0737
Epoch 272/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9793 - loss: 0.0737
Epoch 273/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9868 - loss: 0.0602
Epoch 274/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9743 - loss: 0.0668
Epoch 275/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9740 - loss: 0.0740
Epoch 276/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9678 - loss: 0.0858
Epoch 277/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9759 - loss: 0.0754
Epoch 278/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9794 - loss: 0.0749
Epoch 279/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9849 - loss: 0.0657
Epoch 280/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9781 - loss: 0.0699
Epoch 281/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9838 - loss: 0.0648
Epoch 282/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - ac

22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9858 - loss: 0.0538
Epoch 339/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9890 - loss: 0.0583
Epoch 340/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9938 - loss: 0.0459
Epoch 341/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9887 - loss: 0.0563
Epoch 342/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9897 - loss: 0.0481
Epoch 343/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9911 - loss: 0.0563
Epoch 344/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9752 - loss: 0.0662
Epoch 345/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9886 - loss: 0.0544
Epoch 346/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9822 - loss: 0.0632
Epoch 347/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9920 - loss: 0.0442
Epoch 348/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9867 - loss: 0.0508
Epoch 349/350
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - ac

C:\Users\MOHAMMAD SAIZE ALI\AppData\Local\Temp\ipykernel_13868\2018821962.py:123: RuntimeWarning: divide by zero encountered in log2
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
C:\Users\MOHAMMAD SAIZE ALI\AppData\Local\Temp\ipykernel_13868\2018821962.py:123: RuntimeWarning: invalid value encountered in multiply
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step


In [9]:
print("Accuracy of model on training data",model.evaluate(train_images,train_labels)[1])


22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.9874 - loss: 0.0572
Accuracy of model on training data 0.9871428608894348


In [10]:
import pandas as pd
test_labels=[]
for i in range(0,len(predictions)):
    # Convert predictions above 0.5 to non-raveling and rest to raveling class     
    if predictions[i]<0.5:
        test_labels.append("Non_raveling")
    else:
        test_labels.append("Raveling")
#Storing filename and labels in dataframe     
df = pd.DataFrame({'filename': filenames, 'class': test_labels})
#Changing dataframe to csv file
df.to_csv('hzpredictionsAnn22103025Final.csv', index=False)    
######################################END OF CODE#######################################################

In [11]:
# CODE implementing LOGISTIC REGRESSION for binary classification

# Importing all the required libraries


import pandas as pd
import os
import numpy as np
import cv2
from sklearn.linear_model import LogisticRegression
# Load and preprocess the training data
train_raveling = os.listdir('train/Raveling')
train_non_raveling = os.listdir('train/Non_raveling')
train_images = []
train_labels = []

for img in train_raveling:
    image = cv2.imread('train/Raveling/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w
    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    # Calculating features as given in the paper 
    
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)

    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    train_images.append(features)
    train_labels.append(1)  #label raveling as 1

for img in train_non_raveling:
    image = cv2.imread('train/Non_raveling/' + img)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w
    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)

    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    train_images.append(features)
    train_labels.append(0)  #label non-raveling as 0
train_images = np.array(train_images)
train_labels = np.array(train_labels) 
import numpy as np
from sklearn.impute import SimpleImputer

# removing NaN error
imp = SimpleImputer(strategy='mean')
train_images = imp.fit_transform(train_images)
# Train the Logistic Regression model
model = LogisticRegression(max_iter=1000)
model.fit(train_images, train_labels)
#reading test images:
test_images=[]
filenames=[]
for i in range(1, 301):
    image = cv2.imread('test/' + str(i) + '.jpg')
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    h, w, _ = image.shape
    total_pixels = h * w
    mean = np.mean(image, axis=(0, 1))
    std = np.std(image, axis=(0, 1))
    skewness = np.mean((image - mean) ** 3, axis=(0, 1)) / (std ** 3)
    kurtosis = np.mean((image - mean) ** 4, axis=(0, 1)) / (std ** 4) 
    entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
    rng = np.max(image, axis=(0, 1)) - np.min(image, axis=(0, 1))
    first_order_histogram = []
    total_pixels = image.shape[0] * image.shape[1]
    first_order_histogram = []
    for c in range(3):
        n_c = np.sum(image[:, :, c])
        Pc_I = n_c / total_pixels
        first_order_histogram.append(Pc_I)

    mean_mod = mean * first_order_histogram
    std_mod = std * first_order_histogram
    skewness_mod = skewness * first_order_histogram
    kurtosis_mod = kurtosis * first_order_histogram
    entropy_mod = entropy * first_order_histogram
    rng_mod = rng * first_order_histogram
    features = np.concatenate([mean_mod, std_mod, skewness_mod, kurtosis_mod, entropy_mod, rng_mod])
    test_images.append(features)
    filenames.append(str(i) + '.jpg')
test_labels2=[]
test_images = np.array(test_images)
test_images=imp.fit_transform(test_images)
#predicting class of test image using our developed model:
test_labels = model.predict(test_images)
for i in range(len(test_labels)):
    if test_labels[i]==0:
        test_labels2.append("Non_raveling")
# #         test_labels2[i]="Non_raveling"
    elif test_labels[i]==1:
        test_labels2.append("Raveling")
#Checking order of accuracy of train data:
accuracy = model.score(train_images, train_labels)
print("Accuracy on training data:", accuracy)
df = pd.DataFrame({'filename': filenames, 'class': test_labels2})
#Changing dataframe to csv file
df.to_csv('predictionsLR1.csv', index=False)


C:\Users\MOHAMMAD SAIZE ALI\AppData\Local\Temp\ipykernel_13868\3170120823.py:26: RuntimeWarning: divide by zero encountered in log2
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
C:\Users\MOHAMMAD SAIZE ALI\AppData\Local\Temp\ipykernel_13868\3170120823.py:26: RuntimeWarning: invalid value encountered in multiply
  entropy = -np.sum(np.log2(image / 256) * (image / 256), axis=(0, 1))
C:\Users\MOHAMMAD SAIZE ALI\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\MOHAMMAD SAIZE ALI\AppData\Local\Temp\ipykernel_13868\3

Accuracy on training data: 0.7971428571428572
